In [8]:
# define a model using pytorch
# a pretrained model adding layers to classify the 4 classes of the images
import torch.nn as nn
from torchvision import models
from torchvision.models.resnet import ResNet18_Weights
from torchvision.models.vgg import VGG16_Weights, VGG11_Weights


class MIDmodel(nn.Module):
    def __init__(self):
        super(MIDmodel, self).__init__()
        # first pretrained
        self.pretrained = models.squeezenet1_1(pretrained=True)
        # add layer(s)
        self.fc1 = nn.Linear(1000, 128)
        self.fc2 = nn.Linear(128, 4)
        self.dropout = nn.Dropout(0.7)

    def forward(self, x):
        x = self.pretrained(x)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [9]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pandas as pd
from PIL import Image
import os

class ImageDataset(Dataset):
    def __init__(self, root_dir, patients_ids, patients_df, transform=None):
        # select the rows of the dataframe that correspond to the patients in the list
        self.patients_df = patients_df[patients_df["patient_id"].isin(patients_ids)]
        self.root_dir = root_dir
        self.idxs = set()
        if transform:
            self.transform = transform
            # duplicate the patients_df, so the network will see each image twice, one standard and one transformed
            # self.patients_df = pd.concat([self.patients_df] * 2, ignore_index=True)
        else:
            self.transform = None
               
            
    def __len__(self):
        return len(self.patients_df)

    def __getitem__(self, idx):
        # get idx row of the dataframe
        row = self.patients_df.iloc[idx]
        # path = root_dir/patientid_examid_spot_framenumber_score.png
        path = os.path.join(self.root_dir, row["patient_id"] + "_" + row["exam_id"] + "_" + row["spot"] + "_" + row["frame_number"] + "_" + row["score"] + ".png")
        image = Image.open(path)
        label = row["score"]
        # if the image is grayscale, convert it to RGB (used to test with the smaller dataset)
        # if image.mode != "RGB":
        #     image = image.convert("RGB")
               
        if self.transform:
            image = self.transform(image)
            # if path not in self.idxs: # if it's the first time we see the image, add its path to the set
            #     self.idxs.add(path)
            # else: # if it's the second time we see the image, apply the transform and remove its path from the set 
            #     image = self.transform(image)
            #     self.idxs.remove(path)
        # in any case, transform to tensor
        image = transforms.ToTensor()(image)
        image = transforms.Resize((224, 224))(image)
        # Finally the values are first rescaled to [0.0, 1.0] and then normalized using mean=[0.485, 0.456, 0.406] and std=[0.229, 0.224, 0.225].
        image = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(image)
        return image, int(label)

In [10]:
import pandas as pd
import torch
import glob
data_dir = "images/"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
images_paths = glob.glob(f"{data_dir}*.png", recursive=True)
# images are named as: patientid_examid_spotnumber_framenumber_score.png
# create a dataframe with the data removing "images/"
images_df = pd.DataFrame([path[len(data_dir):-4].split("_") for path in images_paths], columns=["patient_id", "exam_id", "spot", "frame_number", "score"])
images_df["score"] = images_df["score"].astype(str)
images_df["frame_number"] = images_df["frame_number"].astype(str)
images_df["spot"] = images_df["spot"].astype(str)
images_df["patient_id"] = images_df["patient_id"].astype(str)
images_df["exam_id"] = images_df["exam_id"].astype(str)
patients_ids = set(images_df["patient_id"])
patients_ids = list(patients_ids)

# select the 8 patients for training based on the most balanced distribution of the scores
import itertools
combs = list(itertools.combinations(patients_ids, 8))
stds = []
for i, c in enumerate(combs):
    stds.append((images_df[images_df["patient_id"].isin(c)].groupby("score").count()["patient_id"].std(), i))

train_patients = [x for x in combs[min(stds)[1]]]
print(train_patients)
print(images_df[images_df["patient_id"].isin(train_patients)].groupby("score").count()["patient_id"])
test_patients = [x for x in patients_ids if x not in train_patients]
print(test_patients)
print(images_df[images_df["patient_id"].isin(test_patients)].groupby("score").count()["patient_id"])


['1051', '1047', '1052', '1069', '1048', '1067', '1017', '1050']
score
0    7888
1    7540
2    7189
3    7592
Name: patient_id, dtype: int64
['1045', '1068', '1066']
score
0    6625
1    1721
2    5535
3    3631
Name: patient_id, dtype: int64


In [11]:
# transformations from the paper Deep Learning for Classification and Localization of COVID-19 Markers in Point-of-Care Lung Ultrasound
# were each activated on the image-label pair with a probability of 0.33. The set of augmentation functions, each applied with a randomly sampled strength bounded by a set maximum, consists of:
# - affine transformations (translation (max. ±15%), rotation (max. ±15◦), scaling (max. ±45%), and shearing (max. ±4.5◦))
# - multiplication with a constant (max. ±45%)
# - Gaussian blurring (σmax = 3/4 )
# - contrast distortion (max. ±45%)
# - horizontal flipping (p = 0.5)
# - additive white Gaussian noise (σmax = 0.015)
transformations = transforms.Compose([
    transforms.RandomApply([transforms.RandomAffine(degrees=15, translate=(0.15, 0.15), scale=(0.55, 1.45), shear=4.5)], p=1/3),
    transforms.RandomApply([transforms.ColorJitter(brightness=0.45, contrast=0.45, saturation=0.45, hue=0.45)], p=1/3),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=3, sigma=(0.75))], p=1/3),
    transforms.RandomApply([transforms.RandomHorizontalFlip(p=0.5)], p=1/3)
    ])

In [12]:
from torch import optim
from tqdm import tqdm

def train_model(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    for inputs, labels in tqdm(dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_corrects/ len(dataloader.dataset)
    return epoch_loss, epoch_acc

def test_model(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_corrects / len(dataloader.dataset)
    return epoch_loss, epoch_acc


In [13]:
data_dir = "images/"
num_epochs = 10
learning_rate = 0.00001
batch_size = 8

model = MIDmodel()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


train_dataset = ImageDataset(data_dir, train_patients, images_df, transformations)
test_dataset = ImageDataset(data_dir, test_patients, images_df)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

c:\Users\Davide\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Davide\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [14]:
results_train = []
results_test = []

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    train_loss, train_acc = train_model(model, train_dataloader, optimizer, criterion, device)
    results_train.append([train_loss, train_acc])
    print(f"Training Loss: {train_loss:.4f} Training Accuracy: {train_acc:.4f}")
    test_loss, test_acc = test_model(model, test_dataloader, criterion, device)
    results_test.append([test_loss, test_acc])
    print(f"Test Loss: {test_loss:.4f} Test Accuracy: {test_acc:.4f}")
    if epoch % 5 == 0:
        torch.save(model.state_dict(), f"MIDmodel_{epoch}.pt")
    if epoch+1 % 2 == 0:
        learning_rate = learning_rate/2
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

print("Train: ", results_train)
print("Test: ", results_test)

# save the model
torch.save(model.state_dict(), f"MIDmodel.pt")

Epoch 0


100%|██████████| 3777/3777 [16:55<00:00,  3.72it/s]


Training Loss: 1.3769 Training Accuracy: 0.3043


100%|██████████| 2189/2189 [04:03<00:00,  8.99it/s]


Test Loss: 1.6885 Test Accuracy: 0.3751
Epoch 1


...

Training Loss: 0.5421 Training Accuracy: 0.7969


100%|██████████| 2189/2189 [04:10<00:00,  8.73it/s]


Test Loss: 1.9275 Test Accuracy: 0.4536
Epoch 4


  3%|▎         | 98/3777 [00:26<16:38,  3.68it/s]


KeyboardInterrupt: 

In [ ]:
# load the last model
model = MIDmodel()
model.load_state_dict(torch.load("MIDmodel.pt"))
model.to(device)
model.eval()

# create two dataframe to save the wrong and right predictions
import pandas as pd
# TODO: better define the dataframe
wrong_preds = pd.DataFrame(columns=["label", "0", "1", "2", "3"])
right_preds = pd.DataFrame(columns=["label", "0", "1", "2", "3"])

# iterate over test dataset and save softmax outputs
for i in tqdm(range(len(test_dataset))):
    image, label = test_dataset[i]
    image = image.unsqueeze(0)
    image = image.to(device)
    output = model(image)
    _, pred = torch.max(output, 1)
    # save the label and the softmax outputs to the dataframe using concat
    output = (output.to("cpu")).detach().numpy()
    if pred != label:
        wrong_preds = pd.concat([wrong_preds, pd.DataFrame({"label": [label], "0": [output[0][0]], "1": [output[0][1]], "2": [output[0][2]], "3": [output[0][3]]})])
    else:
        right_preds = pd.concat([right_preds, pd.DataFrame({"label": [label], "0": [output[0][0]], "1": [output[0][1]], "2": [output[0][2]], "3": [output[0][3]]})])

wrong_preds = wrong_preds.reset_index(drop=True)
wrong_preds.to_json("wrong_preds.json")

right_preds = right_preds.reset_index(drop=True)
right_preds.to_json("right_preds.json")

In [ ]:
# load json from wrong_preds.json as a dataframe
wrong_preds = pd.read_json("wrong_preds.json")

values = [0 , 0, 0, 0]
for row in wrong_preds.iterrows():
    # sum the value of the column 0 to 3
    values[0] += row[1][0]
    values[1] += row[1][1]
    values[2] += row[1][2]
    values[3] += row[1][3]

# divide the sum of the column by the number of rows
values = [x / len(wrong_preds) for x in values]
print(values)
from numpy import mean
print(mean(values))

# load json from right_preds.json as a dataframe
right_preds = pd.read_json("right_preds.json")

values = [0 , 0, 0, 0]
for row in right_preds.iterrows():
    # sum the value of the column 0 to 3
    values[0] += row[1][0]
    values[1] += row[1][1]
    values[2] += row[1][2]
    values[3] += row[1][3]

# divide the sum of the column by the number of rows
values = [x / len(right_preds) for x in values]
print(values)
print(mean(values))


t-SNE to visualize

In [ ]:
# impiort TNSE
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

tsne = TSNE(n_components=2, random_state=0, verbose=1)
tsne_obj = tsne.fit_transform(right_preds[["0", "1", "2", "3"]])

# plot the result
plt.scatter(tsne_obj[:, 0], tsne_obj[:, 1], c=right_preds["label"])
plt.show()
